In [2]:
import duckdb

import pandas as pd

In [4]:
sql_query = '''
SELECT * FROM bronze_parking_violations LIMIT 3
'''
with duckdb.connect('data/nyc_parking_violations.db') as con:
    display(con.sql(sql_query).df())

,summons_number,registration_state,plate_type,issue_date,violation_code,vehicle_body_type,vehicle_make,issuing_agency,vehicle_expiration_date,violation_location,violation_precinct,issuer_precinct,issuer_code,issuer_command,issuer_squad,violation_time,violation_county,violation_legal_code,vehicle_color,vehicle_year
0,9010912681,CA,PAS,2022-10-11,17,SUBN,FORD,T,20220788,50.0,50,50,365910,T201,D,0825A,BX,NaN,BLACK,0
1,4858762841,NY,PAS,2023-08-21,36,4DSD,HONDA,V,0,NaN,0,0,0,None,None,1036A,BK,True,GY,2003
2,4854645684,FL,PAS,2023-07-26,36,UT,BMW,V,0,NaN,0,0,0,None,None,0126P,QN,True,WHI,2022


In [12]:
sql_query_import_1='''
CREATE OR REPLACE TABLE parking_violation_codes AS
SELECT*
FROM read_csv_auto('data/dof_parking_violation_codes.csv',normalize_names=True)'''
sql_query_import_2='''
CREATE OR REPLACE TABLE parking_violations_2023 AS
SELECT*
FROM read_csv_auto('data/parking_violations_issued_fiscal_year_2023_sample.csv',normalize_names=True)'''

with duckdb.connect('data/nyc_parking_violations.db') as con:
  con.sql(sql_query_import_1)
  con.sql(sql_query_import_2)
